In [1]:
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent

load_dotenv()

model_client = OpenAIChatCompletionClient(
    model="gpt-4.1-mini",
    api_key=os.getenv("OPENAI_API_KEY")
)

#direct llm call
response = await model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(response)
print('----------------------------------')

#build agent
chat_assistant = AssistantAgent(name="chat_agent", model_client=model_client)
result = await chat_assistant.run(task="tell something I did not know about the sky")

print(result.messages[-1].content)
print('----------------------------------')


finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=7) cached=False logprobs=None thought=None
----------------------------------
Here’s an interesting fact you might not know about the sky: The sky is not actually blue; it only appears blue to us during the day because of Rayleigh scattering. This scattering causes shorter blue wavelengths of sunlight to scatter in all directions more than other colors, making the sky look blue. However, if you were standing on the Moon or in space, the sky would appear completely black, even during the day, because there is no atmosphere to scatter the sunlight. 

Would you like to know more fascinating sky facts?
----------------------------------


In [49]:
history_assistant = AssistantAgent(
    name = 'history_expert',
    model_client=model_client,
    description='A knowledgeable assistant with expertise in world history',
    system_message='You are a history expert with deep knowledge of world history. Provide detailed and accurate answers about historical events, figures and timelines'
)

async def test_history_expert():
    result = await history_assistant.run(task = 'Who was the first President of USA?')
    print(result.messages[-1].content)

await test_history_expert()
await model_client.close()

The first President of the United States was George Washington. He served as the first president from April 30, 1789, to March 4, 1797. Washington was a key figure in the American Revolutionary War as the Commander-in-Chief of the Continental Army and later played a crucial role in the framing of the U.S. Constitution. His presidency set many precedents for the new nation, including the formation of the Cabinet and the tradition of serving only two terms.


## Model from Open Router - It gives api key and access to many Models(including free one's)

In [1]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

def check_my_models():
    api_key = os.getenv("OPEN_ROUTER_API_KEY")
    
    if not api_key:
        print("❌ API key not found in .env file")
        return
    
    print(f"🔑 Using key: {api_key[:20]}...")
    print("\n🔄 Checking OpenRouter...\n")
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    try:
        # Get all models
        response = requests.get(
            "https://openrouter.ai/api/v1/models",
            headers=headers,
            timeout=10
        )
        
        if response.status_code == 200:
            models = response.json()['data']
            
            # Separate free and paid
            free_models = [m for m in models if ':free' in m['id']]
            paid_models = [m for m in models if ':free' not in m['id']]
            
            print(f"✅ Connected successfully!\n")
            print(f"📊 Total models: {len(models)}")
            print(f"🆓 Free models: {len(free_models)}")
            print(f"💰 Paid models: {len(paid_models)}\n")
            
            print("=" * 60)
            print("🆓 FREE MODELS YOU CAN USE:")
            print("=" * 60)
            
            if free_models:
                for i, model in enumerate(free_models[:15], 1):  # Show first 15
                    print(f"\n{i}. {model['id']}")
                    print(f"   Context: {model.get('context_length', 'N/A'):,} tokens")
                    if model.get('pricing'):
                        print(f"   Rate: {model.get('pricing', {}).get('prompt', 'Free')}")
                
                if len(free_models) > 15:
                    print(f"\n... and {len(free_models) - 15} more free models")
            else:
                print("⚠️ No free models found!")
                print("You may need to enable data sharing at:")
                print("https://openrouter.ai/settings/privacy")
            
            print("\n" + "=" * 60)
            print("💡 RECOMMENDED MODELS TO TRY:")
            print("=" * 60)
            
            recommended = [
                "google/gemini-2.0-flash-exp:free",
                "qwen/qwen-2-7b-instruct:free",
                "meta-llama/llama-3.2-3b-instruct:free",
                "mistralai/mistral-7b-instruct:free"
            ]
            
            available_recommended = [m for m in free_models if m['id'] in recommended]
            
            for model in available_recommended:
                print(f"✅ {model['id']}")
            
            for rec in recommended:
                if rec not in [m['id'] for m in available_recommended]:
                    print(f"❌ {rec} (not available)")
            
        else:
            print(f"❌ Error: {response.status_code}")
            print(f"Response: {response.text}")
            
            if response.status_code == 401:
                print("\n⚠️ Authentication failed!")
                print("1. Check your API key is correct")
                print("2. Make sure it starts with 'sk-or-v1-'")
                print("3. Generate a new key at: https://openrouter.ai/keys")
    
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n💡 Try:")
        print("1. pip install requests")
        print("2. Check internet connection")
        print("3. Verify API key in .env file")

# Run it
check_my_models()

🔑 Using key: sk-or-v1-bdaa7794429...

🔄 Checking OpenRouter...

✅ Connected successfully!

📊 Total models: 343
🆓 Free models: 46
💰 Paid models: 297

🆓 FREE MODELS YOU CAN USE:

1. nvidia/nemotron-nano-12b-v2-vl:free
   Context: 128,000 tokens
   Rate: 0

2. minimax/minimax-m2:free
   Context: 204,800 tokens
   Rate: 0

3. alibaba/tongyi-deepresearch-30b-a3b:free
   Context: 131,072 tokens
   Rate: 0

4. meituan/longcat-flash-chat:free
   Context: 131,072 tokens
   Rate: 0

5. nvidia/nemotron-nano-9b-v2:free
   Context: 128,000 tokens
   Rate: 0

6. deepseek/deepseek-chat-v3.1:free
   Context: 163,800 tokens
   Rate: 0

7. openai/gpt-oss-20b:free
   Context: 131,072 tokens
   Rate: 0

8. z-ai/glm-4.5-air:free
   Context: 131,072 tokens
   Rate: 0

9. qwen/qwen3-coder:free
   Context: 262,000 tokens
   Rate: 0

10. moonshotai/kimi-k2:free
   Context: 32,768 tokens
   Rate: 0

11. cognitivecomputations/dolphin-mistral-24b-venice-edition:free
   Context: 32,768 tokens
   Rate: 0

12. googl

In [3]:
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()

def test_model(model_name):
    api_key = os.getenv("OPEN_ROUTER_API_KEY")
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": "Say 'Hello!' in one word"}
        ]
    }
    
    print(f"🧪 Testing model: {model_name}\n")
    
    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data,
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            message = result['choices'][0]['message']['content']
            print(f"✅ SUCCESS! Model works!")
            print(f"📝 Response: {message}")
        else:
            print(f"❌ FAILED: {response.status_code}")
            print(f"Error: {response.text}")
    
    except Exception as e:
        print(f"❌ Error: {e}")

# Test these models one by one
models_to_test = [
    "google/gemini-2.0-flash-exp:free",
    "qwen/qwen-2-7b-instruct:free",
    "meta-llama/llama-3.2-3b-instruct:free"
]

for model in models_to_test:
    test_model(model)
    print("\n" + "-" * 60 + "\n")

🧪 Testing model: google/gemini-2.0-flash-exp:free

❌ FAILED: 429
Error: {"error":{"message":"Provider returned error","code":429,"metadata":{"raw":"google/gemini-2.0-flash-exp:free is temporarily rate-limited upstream. Please retry shortly, or add your own key to accumulate your rate limits: https://openrouter.ai/settings/integrations","provider_name":"Google AI Studio"}},"user_id":"user_34skxHkEh956UXtaKnaxlvgWMxr"}

------------------------------------------------------------

🧪 Testing model: qwen/qwen-2-7b-instruct:free

❌ FAILED: 404
Error: {"error":{"message":"No endpoints found for qwen/qwen-2-7b-instruct:free.","code":404},"user_id":"user_34skxHkEh956UXtaKnaxlvgWMxr"}

------------------------------------------------------------

🧪 Testing model: meta-llama/llama-3.2-3b-instruct:free

❌ FAILED: 404
Error: {"error":{"message":"No allowed providers are available for the selected model.","code":404}}

------------------------------------------------------------



In [ ]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API Key
load_dotenv()

open_router_model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="google/gemini-2.0-flash-exp:free",
    api_key="",
    model_info={
        "family":'google',
        "vision":False,
        "function_calling":True,
        "json_output":False,
        "structured_output":True
    }
)


assistant_agent1 = AssistantAgent(
    name="helpful_agent",
    model_client=open_router_model_client,
    system_message='You are a helpfull assistant agent'
)

result = await assistant_agent1.run(task='hey how are you today?')
print(result.messages)

AuthenticationError: Error code: 401 - {'error': {'message': 'User not found.', 'code': 401}}